# Getting Started : Basic Inputs and Outputs

If you are here then you have already successfully 

1) Installed the code

2) Downloaded the necessary reference data

3) Added ``export picaso_refdata="/path/to/picaso/reference"`` to ~/.bash_profile

If you have not done these things, please return to [Installation Guilde](https://natashabatalha.github.io/picaso/installation.html)

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import astropy.units as u

#picaso
from picaso import justdoit as jdi 
from picaso import justplotit as jpi
#plotting
from bokeh.io import output_notebook
output_notebook()
from bokeh.plotting import show

Loading BokehJS ...

## Load blank slate

In [2]:
start_case = jdi.inputs()

In order to run the code we need (at the minimum) specific info about the:

- **phase angle** 

- **planet** : gravity

- **star** : temperature, metallicity, gravity

- **atmosphere** : P-T profile, chemical composition, cloud properties (discussed later)

Additionally, we have some optional parameters that will be discussed in later notebooks: 

- **approx** : approximations to rapidly compute reflected light 

- **disco** : number of individual facets you want to compute before integrating to 1D spectrum (think disco ball) 

- **opacities** : keeping track of opacity files used 

- **test_mode** : used to do benchmark testing 

## Set Planet & Star Properties 

In [3]:
#phase angle 
start_case.phase_angle(0) #radians

#define gravity
start_case.gravity(gravity=25, gravity_unit=u.Unit('m/(s**2)')) #any astropy units available

#define star 
start_case.star(5000,0,4.0) #pysynphot database, temp, metallicity, logg 

## Set Atmospheric Composition

There are different options for setting atmospheric composition. 

    1) Specifying a file path to model run 
    2) Give arbitrary pressure, temperature and composition directly as a dictionary input 


### Option 1) Specify file path 

Below, I am loading in a profile path for Jupiter that should be included in your reference data

In [4]:
print(jdi.jupiter_pt()) #should return the path to your reference data

/Users/natashabatalha/Documents/PICASO/picaso/reference/base_cases/jupiter.pt


In [5]:
start_case.atmosphere(filename=jdi.jupiter_pt(), delim_whitespace=True)

### File format

1) Must have **pressure**(bars), **temperature**(Kelvin), and `case sensitive` molecule names (e.g. TiO, Na, H2O, etc) for mixing ratios (in no particular order)

2) Can specify any necessary key word arguments for pd.read_csv at the end

**PICASO will auto-compute mixing ratios, determine what CIA is neceesary and compute mean molecular weight based on these headers. Take at the preloaded example below**

In [6]:
#to give you an idea
comp_file = pd.read_csv(jdi.jupiter_pt(), delim_whitespace=True) 
#see example below
comp_file.head()

,pressure,temperature,e-,H2,H,H+,H-,VO,TiO,CO2,He,H2O,CH4,CO,NH3,N2,PH3
0,0.000001,150.87,4.500000e-38,0.837,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,0.163,0.000069,0.000466,4.500000e-38,0.000137,5.420000e-17,4.500000e-38
1,0.000001,149.68,4.500000e-38,0.837,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,0.163,0.000035,0.000466,4.500000e-38,0.000137,1.580000e-17,4.500000e-38
2,0.000002,148.40,4.500000e-38,0.837,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,0.163,0.000017,0.000466,4.500000e-38,0.000137,4.370000e-18,4.500000e-38
3,0.000003,147.00,4.500000e-38,0.837,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,0.163,0.000008,0.000466,4.500000e-38,0.000137,1.140000e-18,4.500000e-38
4,0.000004,145.46,4.500000e-38,0.837,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,4.500000e-38,0.163,0.000004,0.000466,4.500000e-38,0.000137,2.740000e-19,4.500000e-38


## Create 1D Spectrum

Let's create our first spectrum of Jupiter's reflected light at full phase

In [7]:
wno, alb = start_case.spectrum()
show(jpi.spectrum(wno, alb, plot_width=500))

### Option 2) Arbitrary PT and Chemistry

Sometimes for testing (or for atmospheres that we don't fully understand) an isothermal, well-mixed profile is sufficient. If we don't want to load in a full profile, we can give it a simple DataFrame with the info we need. 


In [8]:
start_case.atmosphere( df = pd.DataFrame({'pressure':np.logspace(-6,2,60),
                                                 'temperature':np.logspace(-6,2,60)*0+200,
                                                 "H2":np.logspace(-6,2,60)*0+0.837,
                                                 "He":np.logspace(-6,2,60)*0+0.163,
                                                 "CH4":np.logspace(-6,2,60)*0+0.000466})
                     )

In [9]:
wno_ch4, alb_ch4 = start_case.spectrum()

In [10]:
show(jpi.spectrum(wno_ch4, alb_ch4, plot_width=500))

See how the plot above is much easier to interpret than the one with the full set of molecular input. Here we can clearly see the effects of methane opacity, raman and rayleigh scattering (the next notebook will include a tutorial for more diagnostic plotting)

### Diagnostic help: Sometimes it helps to exclude molecule to see how it is influencing the spectrum

Take a look below

In [11]:
start_case.atmosphere(filename=jdi.jupiter_pt(), exclude_mol='H2O', delim_whitespace=True)

wno_nowater, alb_nowater = start_case.spectrum()
fig = jpi.spectrum(wno, alb, plot_width=500)
fig.line(1e4/wno_nowater, alb_nowater, line_width=2, color='red')
show(fig)